In [1]:
from data.GroundTruth import DeepAccidentDataset
from method.dynamics import InDCBFAttentionDynamics, GroundTruthiDBFDynamics
from method.barriers import InDCBFAttentionBarrier, InDCBFBarrier
from method.trainers import InDCBFTrainer
import torch
import time
import pytorch_lightning as pl
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [8]:
data = DeepAccidentDataset(train_batch_size=32,val_batch_size=32,num_workers=16)
data.setup()
train_dataloader = data.train_dataloader()
test_dataloader = data.test_dataloader()

In [3]:
latent_dim = 16
barrier = InDCBFAttentionBarrier(2,latent_dim=latent_dim)
model = GroundTruthiDBFDynamics(2,"cuda",latent_dim=latent_dim)
trainer = InDCBFTrainer(model,barrier)
checkpoint = torch.load("/root/tf-logs/GroundTruth_InDCBF_42/version_3/checkpoints/last.ckpt")
trainer.load_state_dict(checkpoint['state_dict'])
trainer.to("cuda")

InDCBFTrainer(
  (model): GroundTruthiDBFDynamics(
    (encoder): QuantityPermutationInvariantEncoder(
      (linear): Linear(in_features=16, out_features=16, bias=True)
      (mlp): Sequential(
        (0): Linear(in_features=4, out_features=128, bias=True)
        (1): ReLU()
        (2): Linear(in_features=128, out_features=16, bias=True)
        (3): ReLU()
      )
      (activation): ReLU()
    )
    (ode): NeuralODE(
      (ode_f): Sequential(
        (0): Linear(in_features=16, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
        (3): ReLU()
        (4): Linear(in_features=64, out_features=16, bias=True)
        (5): ReLU()
      )
      (ode_g): Sequential(
        (0): Linear(in_features=16, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
        (3): ReLU()
        (4): Linear(in_features=64, out_features=32, bias=True)
        (5): ReLU()
      )

In [9]:
b_all = []
label_all = []
trainer.eval()
for idx, (i,u,label) in enumerate(test_dataloader):
    i, u = i.to("cuda"), u.to("cuda")
    x,x_tide = trainer.model.simulate(i,u)
    b = trainer.barrier(x).squeeze(-1)
    b_all.append(b.cpu())
    label_all.append(label.squeeze(-1))
import numpy as np
bs = torch.cat(b_all)
labels = torch.cat(label_all)
results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
np.savetxt("./results_indcbf_groundtruth.txt",results)

100%|██████████| 4/4 [00:00<00:00, 175.58it/s]


In [10]:
results = np.loadtxt("./results_indcbf_groundtruth.txt")
regular = results[results[:,-1] == 0,:-1]
collision = results[results[:,-1] == 1,:-1]
acc_regular = (regular > 0).mean()
acc_collision = (collision < 0).mean()
print(acc_regular,acc_collision)

0.9175066312997348 0.7652173913043478


In [11]:
b_all = []
label_all = []
trainer.eval()
for idx, (i,u,label) in enumerate(test_dataloader):
    i, u = i.to("cuda"), u.to("cuda")
    x,x_tide = trainer.model.simulate(i,u)
    b = trainer.barrier(x)
    d_b_safe = torch.autograd.grad(b.mean(),x,retain_graph=True)[0]
    with torch.no_grad():
        f, g = trainer.model.ode(x)
    gu = torch.einsum('btha,bta->bth',g.view(g.shape[0],g.shape[1],f.shape[-1],2),u)
    ascent_value = torch.einsum('bth,bth->bt', d_b_safe, (f + gu)) + b.squeeze(-1)
    b_all.append(ascent_value.cpu())
    label_all.append(label.squeeze(-1))
bs = torch.cat(b_all)
labels = torch.cat(label_all)
results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
np.savetxt("./results_indcbf_grad_groundtruth.txt",results)

100%|██████████| 4/4 [00:00<00:00, 219.59it/s]


In [12]:
results = np.loadtxt("./results_indcbf_grad_groundtruth.txt")
regular = results[results[:,-1] == 0,:-1]
collision = results[results[:,-1] == 1,:-1]
acc_regular = (regular > 0).mean()
acc_collision = (collision < 0).mean()
print(acc_regular,acc_collision)

0.9175066312997348 0.7652173913043478
